In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import datetime
import statistics as stats
import tsfresh.feature_extraction.feature_calculators as fc
from functools import reduce
import math

### Daten einlesen und Spalten entfernen welche nicht benötigt werden.

In [ ]:
reib_daten = pd.read_csv('data_2019-26-11_TRAININGDATA.csv')
achsleistung_daten = pd.read_csv('../Achsleistungen _ Messprotokolle/Ebene 1/Achsleistung-2019-11-20T10-27-03.csv', \
                                 delimiter=';')
messprotokoll_daten = pd.read_csv('../Achsleistungen _ Messprotokolle/Ebene 1/MEAS_PROTOCOL_CSV.CSV', delimiter=';')
messergebnisse_daten = pd.read_csv('../Messergebnisse Reiben/Messergebnisse.csv', skiprows=7, delimiter=';')
achsleistung_columns = ['Timestamp', 'X1', 'Y1', 'Z1', 'SPI1']
messergebnisse_columns = ['Messwert Innenmessschraube', 'Messwert Messtaster', 'Klassifizierung nach Messtaster', \
                          'Klassifizierung nach Messtaster', 'Klassifizierung nach Grenzlehrdorn']
achsleistung_daten = achsleistung_daten[achsleistung_columns]
messergebnisse_daten = messergebnisse_daten[messergebnisse_columns]

### Anpassen und aufbereiten Messprotokolldaten
Da die Maschine die Werte in der Exceltabelle sehr schlecht formatiert - viele Tabulatoren und alle Zahlen sind strings. Zusätzlich werden noch alle null values entfernt.

In [ ]:
messprotokoll_daten = messprotokoll_daten.rename(columns=lambda x: x.strip())
messprotokoll_columns = ['Measured', 'Difference']
messprotokoll_daten = messprotokoll_daten[messprotokoll_columns]

for column, value in enumerate(messprotokoll_columns):
    messprotokoll_daten[value] = messprotokoll_daten[value].str.strip()
    messprotokoll_daten[value] = messprotokoll_daten[value].str.replace(',', '.')
    messprotokoll_daten[value] = messprotokoll_daten[value].apply(pd.to_numeric, errors='coerce')

print(messprotokoll_daten.dtypes)

achsleistung_daten = achsleistung_daten.dropna()
messprotokoll_daten = messprotokoll_daten.dropna()
messergebnisse_daten = messergebnisse_daten.dropna()

### Berechnung Mittelwert und Median für Messprotokolldaten

Hier werden die Daten für das Messprotokoll angepasst. Da hier eine Bohrung aus drei Werten besteht (Spalte Result: X, Y, Diameter), werden alle drei Werte genommen und aggregiert $\rightarrow$ Median + Mittelwert.

Danach wird das Messprotokoll mit den Daten der Messergebnisse gemerged.

In [ ]:
filling_columns = ['Measured Mean', 'Measured Median', 'Difference Mean', 'Difference Median']

messprotokoll_condensed = pd.DataFrame(columns=filling_columns)
measured_avg = []
measured_median = []
diameter_avg = []
diameter_median = []
count = 1
for key, value in messprotokoll_daten.iteritems():
    if key == 'Measured':
        meas = []
        for index, floating in enumerate(value):
            meas.append(floating)
            if (count % 3) == 0:
                mean_meas = stats.mean(meas)
                median_meas = stats.median(meas)
                measured_avg.append(mean_meas)
                measured_median.append(median_meas)
                #print(measured_avg, '\n', measured_median)
                del meas[:]
                
            #print(count % 3 == 0, meas, count)
            if count == 1:
                pass
            
            count += 1

    elif key == 'Difference':
        diff = []
        for index, floating in enumerate(value):
            diff.append(abs(floating))
            if (count % 3) == 0:
                mean_diff = np.mean(diff)
                median_diff = np.median(diff)
                diameter_avg.append(mean_diff)
                diameter_median.append(median_diff)
                del diff[:]
            
            if count == 1:
                pass
            
            count += 1

            

messprotokoll_condensed['Measured Mean'] = measured_avg
messprotokoll_condensed['Measured Median'] = measured_median
messprotokoll_condensed['Difference Mean'] = diameter_avg
messprotokoll_condensed['Difference Median'] = diameter_median

merge_csv_erg_prot = messergebnisse_daten[:81].merge(messprotokoll_condensed, right_index=True , left_index=True)

merge_csv_erg_prot

### Achsleitungsverarbeitung und entfernung Ausreiser?
Hier wird die Achleistung bearbeitet. Diskutierbar ist, ob "Ausreiser" die durch Median * 3 ermittelt werden, entfernt werden müssen oder nicht. Es macht keine Unterschiede in der Anzahl der Daten, jedoch in deren Aggregierten Werten (Max, Mean, Median).

In [ ]:
# delete any row with 0's in SPI
achsleistung_daten = achsleistung_daten[(achsleistung_daten[['SPI1']] != 0).all(axis=1)]

# remove median > 3 for all columns
for key, value in achsleistung_daten.items():
    if achsleistung_daten[key].dtypes == 'float64':
        achsleistung_daten.drop(achsleistung_daten[value > (3 * stats.mean(achsleistung_daten[key]))].index, inplace=True)
        

achsleistung_daten = achsleistung_daten.set_index(['Timestamp'])

In [ ]:
# achsleistung_daten.index = pd.to_datetime(achsleistung_daten.index)
# print(achsleistung_daten.index[-1], achsleistung_daten.index[0])
# time_diff = (achsleistung_daten.index[-1] - achsleistung_daten.index[0])

# resampling_range = time_diff.total_seconds() / 81

# print(math.floor(resampling_range))

# index_series = pd.date_range(start=achsleistung_daten.index[0], end=achsleistung_daten.index[-1], periods=time_diff.total_seconds())

# achsleistung_daten = achsleistung_daten.reindex(achsleistung_daten.index.union(index_series)).interpolate(method='time').reindex(index_series)

# achsleistung_daten
#dateRangeAndAggregate()

### Aufteilung der Daten in gleich große Zeitintervalle und Aggregierung der Daten
Hier wird zunächst die Gesamtzeit zwischen Start und Ende der Achsleistung bestimmt. Diese wird durch 81 geteilt (um einen Zeitraum für eine Bohrung bestimmen zu können), und mit diesem Zeitraum werden die Daten auf verschiedenen Aggregationen - Max, Min, Mean, Median - neu gesampled.
Die berechneten Werte werden dann dem zuvor erstellten DataFrame "merge_csv_erg_prot" hinzugefügt.

Die Datenverarbeitung ist damit abgeschlossen.

In [ ]:
def reduce_df(dataframe):
    reduced_frame = reduce(lambda left,right: pd.merge(left,right,on='Timestamp'), dataframe)
    return reduced_frame

achsleistung_daten.index = pd.to_datetime(achsleistung_daten.index)

In [ ]:
time_diff = (achsleistung_daten.index[-1] - achsleistung_daten.index[0])
resampling_range = (time_diff.total_seconds() / 81)

print('resampling range: '+ str(resampling_range) + '\ntime difference: ' + str(time_diff) + \
      '\ntime of one drill: ' + str((time_diff / resampling_range)))

sampled_mean = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').mean()
sampled_median = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').median()
sampled_max = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').max()
sampled_min = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').min()
#sampled_app_entropy = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').apply(fc.approximate_entropy(achsleistung_daten), 81, r=1.0)
#sampled_std_deviation = fc.standard_deviation(achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').std())

sampled_mean = sampled_mean.rename(
    columns={'X1': 'X1_mean', 'Y1': 'Y1_mean', 'Z1': 'Z1_mean', 'SPI1': 'SPI1_mean'})
sampled_median = sampled_median.rename(
    columns={'X1': 'X1_median', 'Y1': 'Y1_median', 'Z1': 'Z1_median', 'SPI1': 'SPI1_median'})
sampled_max = sampled_max.rename(                         
    columns={'X1': 'X1_max', 'Y1': 'Y1_max', 'Z1': 'Z1_max', 'SPI1': 'SPI1_max'})
sampled_min = sampled_min.rename(                         
    columns={'X1': 'X1_min', 'Y1': 'Y1_min', 'Z1': 'Z1_min', 'SPI1': 'SPI1_min'})
        
        
df_list = [sampled_mean, sampled_median, sampled_max, sampled_min]
df_final = reduce_df(df_list)

df_final = df_final.reset_index()

df_final

In [ ]:
merge_csv_erg_prot = merge_csv_erg_prot.merge(df_final, right_index=True, left_index=True)

In [ ]:
merge_csv_erg_prot